In [1]:
import numpy as np
import keras
from keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split
import json
import os
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import sklearn
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [2]:
from sklearn.metrics import accuracy_score

In [3]:
file_path = './combined_all_interpol_synced_4sec_800samples_prot4.pkl'

dataFrame = pd.read_pickle(file_path)


print(dataFrame.columns)
y = np.array(dataFrame['uid'])
x = None
for col in dataFrame.columns:
    if 'sensordata' in col:
        if x is None:
            x = np.array(dataFrame[col].apply(lambda x: np.array(x)).to_list())
            #print(x_data.shape)
        else:
            x_data = np.array(dataFrame[col].apply(lambda x: np.array(x)).to_list())
            #print(x_data.shape)
            x = np.append(x, x_data , axis=2)
dur = np.array(dataFrame['duration'])
print("Dataset shapes: ")
print(x.shape)
print(dur.shape)
print(y.shape)


def split_features_and_adjust_labels(x, y, window_size=100, overlap=0.4):
    num_samples = x.shape[0]
    num_timesteps = x.shape[1]
    num_features = x.shape[2]

    step_size = int(window_size * (1 - overlap))
    num_segments = int((num_timesteps - window_size) / step_size) + 1

    split_x = np.zeros((num_samples * num_segments, window_size, num_features))
    adjusted_y = np.repeat(y, num_segments)

    segment_index = 0
    for sample_index in range(num_samples):
        for i in range(0, num_timesteps - window_size + 1, step_size):
          split_x[segment_index] = x[sample_index, i : i + window_size, :]
          segment_index += 1

    return split_x, adjusted_y, num_segments

def get_aligned_and_rest(split_x, adjusted_y,num_segments):
    x_aligned = []
    y_aligned = []
    x_other = []
    y_other = []

    for i in range(len(adjusted_y)):
        if i % num_segments == 0:
            x_aligned.append(split_x[i])
            y_aligned.append(adjusted_y[i])
        else:
            x_other.append(split_x[i])
            y_other.append(adjusted_y[i])
    return np.array(x_aligned), np.array(y_aligned), np.array(x_other), np.array(y_other)


split_x, adjusted_y,num_segments  = split_features_and_adjust_labels(x,y,window_size=100,overlap=0)
print(split_x.shape, adjusted_y.shape)
x_aligned, y_aligned, x_other, y_other = get_aligned_and_rest(split_x,adjusted_y, num_segments)


y_aligned = y_aligned.astype(str)
y_other = y_other.astype(str)
x = x_aligned
y = y_aligned


class_label_encoder = LabelEncoder()
scaler = StandardScaler()

y = class_label_encoder.fit_transform(y)
y_other = class_label_encoder.transform(y_other)
n_classes = class_label_encoder.classes_.shape[0]

print("train-test shapes: ")
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
x_train = scaler.fit_transform(x_train.reshape(-1, x_train.shape[-1])).reshape(x_train.shape)
x_test = scaler.transform(x_test.reshape(-1, x_test.shape[-1])).reshape(x_test.shape)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

x_other_scaled = scaler.transform(x_other.reshape(-1, x_other.shape[-1])).reshape(x_other.shape)


print(x_other_scaled.shape, y_other.shape)

Index(['uid', 'sensordata.ACC', 'sensordata.GYR', 'sensordata.GRV',
       'sensordata.MAG', 'sensordata.ROT', 'duration'],
      dtype='object', name='stype')
Dataset shapes: 
(332, 800, 17)
(332,)
(332,)
(2656, 100, 17) (2656,)
train-test shapes: 
(232, 100, 17) (100, 100, 17) (232,) (100,)
(232, 100, 17) (100, 100, 17) (232,) (100,)
(2324, 100, 17) (2324,)


In [55]:
from keras import losses,Model
from keras.models import load_model
latent_dim = 64
MLength = 100
dims = 17


MPATH = './new_autoencoder_model_target_0'
input_shape = all_x_train.shape[1:]

class SaveModelCheckpoint(keras.callbacks.Callback):
    def __init__(self, filepath, interval=10):
        super(SaveModelCheckpoint, self).__init__()
        self.filepath = filepath
        self.interval = interval

    def on_train_end(self, logs=None):
        filepath = self.filepath
        # Create the directory if it doesn't exist
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        print(f"Saving history to {filepath}")
        with open(self.filepath+'.json', 'a') as f:
            json.dump(self.model.history.history, f, indent=4)

# Create an instance of the custom callback
save_checkpoint_callback = SaveModelCheckpoint(MPATH, interval=10)

callbacks = [keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True), save_checkpoint_callback]

@keras.saving.register_keras_serializable()
class Autoencoder(Model):
  def __init__(self, latent_dim, mLength, dims, **kwargs):
      super(Autoencoder, self).__init__(**kwargs)
      self.latent_dim = latent_dim
      self.mLength = mLength
      self.dims = dims
      self.encoder = tf.keras.Sequential([
          layers.Flatten(),
          layers.Dense(self.latent_dim, activation='relu'),
      ])
      self.decoder = tf.keras.Sequential([
          layers.Dense(self.mLength * self.dims, activation='linear'),
          layers.Reshape((self.mLength, self.dims)),
      ])

  def call(self, x):
      encoded = self.encoder(x)
      decoded = self.decoder(encoded)
      return decoded

  def get_config(self):
      # Include all parameters needed for re-creation
      config = super(Autoencoder, self).get_config()
      config.update({
          "latent_dim": self.latent_dim,
          "mLength": self.mLength,
          "dims": self.dims,
      })
      return config

  @classmethod
  def from_config(cls, config):
      # Return an instance of the model
      return cls(**config)




autoencoder = Autoencoder(latent_dim,MLength,dims)
autoencoder.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                    loss=losses.MeanSquaredError())
autoencoder.summary()

Model: "autoencoder_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_12 (Sequential)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_13 (Sequential)      │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
MPATH = './new_autoencoder_model'
input_shape = x_train.shape[1:]
class SaveModelCheckpoint(keras.callbacks.Callback):
    def __init__(self, filepath, interval=10):
        super(SaveModelCheckpoint, self).__init__()
        self.filepath = filepath
        self.interval = interval
    def on_train_end(self, logs=None):
        filepath = self.filepath.format(epoch="0")
        # Create the directory if it doesn't exist
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        print(f"Saving history to {filepath}")
        with open(self.filepath+'.json', 'a') as f:
            json.dump(self.model.history.history, f, indent=4)

# Create an instance of the custom callback
save_checkpoint_callback = SaveModelCheckpoint(MPATH +'_{epoch}', interval=10)

callbacks = [keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True), save_checkpoint_callback]

In [60]:
with tf.device('/GPU:0'):
    autoencoder.fit(
    x_train,
    x_train,
    validation_split=0.2,
    epochs=100,
    batch_size=4,
    callbacks=callbacks,
    )

Epoch 1/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2979 - val_loss: 0.7554
Epoch 2/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 978us/step - loss: 1.3137 - val_loss: 0.7364
Epoch 3/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1714 - val_loss: 0.7112
Epoch 4/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 978us/step - loss: 1.0236 - val_loss: 0.6734
Epoch 5/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0300 - val_loss: 0.6237
Epoch 6/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7814 - val_loss: 0.5626
Epoch 7/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7157 - val_loss: 0.5072
Epoch 8/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8747 - val_loss: 0.4533
Epoch 9/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5672 - val_loss: 0.4178
Epoch 10/100
 1/47 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.3950Saving model to ./new_autoencoder_model_10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5945 - val_loss: 0.3845
Epoch 11/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/s